In [1]:
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout, Softmax, Flatten, Activation, BatchNormalization
from tensorflow.keras.models import Sequential, Model

Using TensorFlow backend.


In [2]:
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

In [3]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model.load_weights("D:\\opencv\\vgg_face_weights.h5")

In [ ]:
#Finally, we’ll use previous layer of the output layer for representation. The following usage will give output of that layer.

In [7]:
vgg= Model(inputs = model.layers[0].input, outputs=model.layers[-2].output)

In [9]:
model.layers[0].input

<tf.Tensor 'zero_padding2d_input:0' shape=(?, 224, 224, 3) dtype=float32>

In [10]:
model.layers[-2].output

<tf.Tensor 'flatten/Reshape:0' shape=(?, 2622) dtype=float32>

In [12]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [15]:
import numpy as np

In [16]:
def preprocess_image(i_path):
    img=load_img(i_path, target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img, axis=0)
    img=preprocess_input(img)
    return img

In [17]:
image1 = vgg.predict(preprocess_image('img1.jpg'))[0,:]
image2 = vgg.predict(preprocess_image('img2.jpg'))[0,:]

In [18]:
image2

array([ 2.9284127 , -0.2610899 , -0.18327881, ..., -2.2216997 ,
        0.25767916,  1.6377125 ], dtype=float32)

In [19]:
image1

array([ 1.9408672 ,  1.8806499 ,  0.93208593, ..., -4.3586454 ,
       -2.5393648 ,  2.9280612 ], dtype=float32)

In [24]:
def findCosineDistance(source_representation, test_representation):
    a=np.matmul(np.transpose(source_representation), test_representation)
    b=np.sum(np.multiply(source_representation,source_representation))
    c=np.sum(np.multiply(test_representation, test_representation))
    return 1-(a/np.sqrt(b)*np.sqrt(c))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance= source_representation-test_representation
    euclidean_distance= np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance= np.sqrt(euclidean_distance)
    return euclidean_distance

In [25]:
epsilon = 0.40 ##cosine similarity
#epsilon = 120 #euclidean distance

In [28]:
def verify_face(img1, img2):
    image1 = vgg.predict(preprocess_image('img1.jpg'))[0,:]
    image2 = vgg.predict(preprocess_image('img2.jpg'))[0,:]
    
    cosine_distance= findCosineDistance(image1, image2)
    euclidean_distance=findEuclideanDistance(image1,image2)
    if(cosine_distance < epsilon):
        print("verified: face_matches")
    else:
        print("unverified: face_doesnot_matches")

In [29]:
verify_face("img1.jpg", "img2.jpg")

verified: face_matches


In [30]:
import cv2